In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('.\\input\\uci-secom.csv')
print(data.shape)
print(data.head(5))
import os

(1567, 592)
                  Time        0        1          2          3       4      5  \
0  2008-07-19 11:55:00  3030.93  2564.00  2187.7333  1411.1265  1.3602  100.0   
1  2008-07-19 12:32:00  3095.78  2465.14  2230.4222  1463.6606  0.8294  100.0   
2  2008-07-19 13:17:00  2932.61  2559.94  2186.4111  1698.0172  1.5102  100.0   
3  2008-07-19 14:43:00  2988.72  2479.90  2199.0333   909.7926  1.3204  100.0   
4  2008-07-19 15:22:00  3032.24  2502.87  2233.3667  1326.5200  1.5334  100.0   

          6       7       8  ...       581     582     583     584      585  \
0   97.6133  0.1242  1.5005  ...       NaN  0.5005  0.0118  0.0035   2.3630   
1  102.3433  0.1247  1.4966  ...  208.2045  0.5019  0.0223  0.0055   4.4447   
2   95.4878  0.1241  1.4436  ...   82.8602  0.4958  0.0157  0.0039   3.1745   
3  104.2367  0.1217  1.4882  ...   73.8432  0.4990  0.0103  0.0025   2.0544   
4  100.3967  0.1235  1.5031  ...       NaN  0.4800  0.4766  0.1045  99.3032   

      586     587     588 

In [2]:
data.isnull().any().any()

True

In [3]:
d = data.isnull().sum()
j = []
for i in d.keys():
    if(d[i] >900):
        print(i, d[i])
        j.append(i)

85 1341
109 1018
110 1018
111 1018
157 1429
158 1429
220 1341
244 1018
245 1018
246 1018
292 1429
293 1429
358 1341
382 1018
383 1018
384 1018
492 1341
516 1018
517 1018
518 1018
578 949
579 949
580 949
581 949


In [4]:
data.drop(j, axis = 1, inplace = True)

In [5]:
data.replace(np.nan, 0, inplace = True)
#from sklearn.preprocessing import Imputer
#imputer = Imputer(missing_values = np.nan, strategy = 'mean')
data.isnull().any().any()

False

In [6]:
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, average_precision_score

In [8]:
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [9]:
data['Pass/Fail'].value_counts()#-1 is pass and 1 is fail

-1    1463
 1     104
Name: Pass/Fail, dtype: int64

In [10]:
x = data.iloc[:, 1:-1]
y = data.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 0)

In [11]:
print(x_train.shape)
print(x_test.shape)
print(data.shape)

(1096, 566)
(471, 566)
(1567, 568)


In [12]:
(data['Pass/Fail'] == 1).index


RangeIndex(start=0, stop=1567, step=1)

In [13]:
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)

In [14]:
lr = LogisticRegression(C = 1000, penalty = 'l2')
lr.fit(x_train_std, y_train)
y_pred_lr = lr.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_lr, y_test))
cm = confusion_matrix(y_pred_lr, y_test)
print('precision_score', average_precision_score(y_pred_lr, y_test))
print('recall_score', recall_score(y_pred_lr, y_test))
print('f1_score',f1_score(y_pred_lr,y_test))

accuracy score 0.851380042462845
precision_score 0.11374514755210494
recall_score 0.07692307692307693
f1_score 0.10256410256410257


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
y_test[y_test == -1].sum()

-445

In [16]:
isolation = IsolationForest()
isolation.fit(x_train_std, y_train)
y_pred_iso = isolation.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_iso, y_test))
cmiso = confusion_matrix(y_pred_iso, y_test)
print('precision_score', average_precision_score(y_pred_iso, y_test))
print('recall_score', recall_score(y_pred_iso, y_test))
print('f1_score',f1_score(y_pred_iso,y_test))

accuracy score 0.055201698513800426
precision_score 1.0
recall_score 0.055201698513800426
f1_score 0.10462776659959759


In [17]:
cmiso

array([[  0,   0],
       [445,  26]], dtype=int64)

In [18]:
print(cmiso[1][1]/ (cmiso[1][1] + cmiso[0][1]))

1.0


In [19]:
forest = RandomForestClassifier()
forest.fit(x_train_std, y_train)
y_pred_rf = forest.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_rf, y_test))
print('confusion_matrix', confusion_matrix(y_pred_rf, y_test))
print('precision_score', average_precision_score(y_pred_rf, y_test))

accuracy score 0.9447983014861996
confusion_matrix [[445  26]
 [  0   0]]
precision_score nan


C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [20]:
y_test[y_test == 1].sum()

26

In [21]:
xgb1 = xgb.XGBClassifier(objective = 'binary:logistic', booster = 'gblinear')
xgb1.fit(x_train_std, y_train)
y_pred_xgb = xgb1.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_xgb, y_test))
print('confusion_matrix', confusion_matrix(y_pred_xgb, y_test))
print('precision_score', average_precision_score(y_pred_xgb, y_test))


accuracy score 0.8726114649681529
confusion_matrix [[403  18]
 [ 42   8]]
precision_score 0.13840274375306222


In [22]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C':[5, 10, 250, 100, 500],
    'penalty':['l1', 'l2']
}
gslr = GridSearchCV(estimator = lr, param_grid = param_grid, n_jobs = -1, cv = 10)
gslr.fit(x_train_std, y_train)
y_pred_gslr = gslr.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_gslr, y_test))
print('confusion_matrix', confusion_matrix(y_pred_gslr, y_test))
print('precision_score', average_precision_score(y_pred_gslr, y_test))

accuracy score 0.8598726114649682
confusion_matrix [[401  22]
 [ 44   4]]
precision_score 0.10623877184386739


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [23]:
print('confusion_matrix', confusion_matrix(y_pred_gslr, y_test))

confusion_matrix [[401  22]
 [ 44   4]]


In [24]:
parameters = { 'max_features':np.arange(5,10),'n_estimators':[500],'min_samples_leaf': [10,50,100,200,500]}

random_grid = GridSearchCV(forest, parameters, cv = 5)
random_grid.fit(x_train_std, y_train)
y_pred_gslr = random_grid.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_gslr, y_test))
print('confusion_matrix', confusion_matrix(y_pred_gslr, y_test))
print('precision_score', average_precision_score(y_pred_gslr, y_test))


accuracy score 0.9447983014861996
confusion_matrix [[445  26]
 [  0   0]]
precision_score nan


C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [25]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train_std, y_train)
y_pred_lda = lda.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_lda, y_test))
print('confusion_matrix', confusion_matrix(y_pred_lda, y_test))
print('precision_score', average_precision_score(y_pred_lda, y_test))
print('recall_score', recall_score(y_pred_lda, y_test))
print('f1_score',f1_score(y_pred_lda,y_test))

accuracy score 0.8832271762208068
confusion_matrix [[411  21]
 [ 34   5]]
precision_score 0.09684166886518675
recall_score 0.1282051282051282
f1_score 0.15384615384615385
